# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
city_df


,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bredasdorp,-34.5322,20.0403,60.80,67,75,2.30,ZA,1613429081
1,1,maceio,-9.6658,-35.7353,78.80,89,40,6.91,BR,1613429389
2,2,lesosibirsk,58.2358,92.4828,0.75,92,100,8.66,RU,1613429390
3,3,khonsa,27.0167,95.5667,54.99,65,55,2.93,IN,1613429390
4,4,tomatlan,19.9333,-105.2500,75.65,48,0,9.37,MX,1613429390
...,...,...,...,...,...,...,...,...,...,...
551,551,desaguadero,-16.5656,-69.0417,51.69,56,84,1.30,PE,1613429499
552,552,tyrma,50.0833,132.1667,-4.45,88,100,4.59,RU,1613429499
553,553,mikhaylovka,50.0600,43.2379,7.00,86,0,3.47,RU,1613429500
554,554,manzanillo,20.3433,-77.1167,80.83,64,15,5.79,CU,1613429500


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
gmaps.configure(api_key=g_key)

In [12]:
coordinates = city_df[["Latitude", "Longitude"]]
humidity = city_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
city_filter = city_df.loc[(city_df["Max Temperature"] < 90) & (city_df["Max Temperature"] > 75) \
                                    & (city_df["Wind Speed"] < 16) \
                                    & (city_df["Cloudiness"] == 0)].dropna()
city_filter

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,tomatlan,19.9333,-105.2500,75.65,48,0,9.37,MX,1613429390
45,45,saint-philippe,-21.3585,55.7679,78.80,69,0,5.75,RE,1613429070
56,56,butaritari,3.0707,172.7902,79.43,79,0,14.36,KI,1613429087
91,91,lazaro cardenas,17.9583,-102.2000,76.19,58,0,10.45,MX,1613429411
103,103,coihaique,-45.5752,-72.0662,75.20,29,0,11.50,CL,1613429098
117,117,saint-pierre,-21.3393,55.4781,78.80,69,0,5.75,RE,1613429414
163,163,avarua,-21.2078,-159.7750,84.20,74,0,4.61,CK,1613429086
256,256,yulara,-25.2406,130.9889,75.20,33,0,9.22,AU,1613429083
271,271,koumac,-20.5667,164.2833,83.79,65,0,2.19,NC,1613429447
279,279,santa lucia,27.9117,-15.5407,75.20,25,0,4.61,ES,1613429449


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
hotel_df = city_filter[["City", "Country", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
4,tomatlan,MX,19.9333,-105.2500,
45,saint-philippe,RE,-21.3585,55.7679,
56,butaritari,KI,3.0707,172.7902,
91,lazaro cardenas,MX,17.9583,-102.2000,
103,coihaique,CL,-45.5752,-72.0662,
117,saint-pierre,RE,-21.3393,55.4781,
163,avarua,CK,-21.2078,-159.7750,
256,yulara,AU,-25.2406,130.9889,
271,koumac,NC,-20.5667,164.2833,
279,santa lucia,ES,27.9117,-15.5407,


In [22]:
# search for hotels
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# loop
for index, row in hotel_df.iterrows():
    #get latitude, longitude from hotel_df
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    
    params["coordinates"] = f"{latitude},{longitude}"
    
    #search for a hotel near by latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #call request
    name_address = requests.get(base_url, params=params)
    
    #jsonify
    name_address = name_address.json()
    
    #grab first listed hotel
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Latitude,Longitude,Hotel Name
4,tomatlan,MX,19.9333,-105.2500,
45,saint-philippe,RE,-21.3585,55.7679,
56,butaritari,KI,3.0707,172.7902,
91,lazaro cardenas,MX,17.9583,-102.2000,
103,coihaique,CL,-45.5752,-72.0662,
117,saint-pierre,RE,-21.3393,55.4781,
163,avarua,CK,-21.2078,-159.7750,
256,yulara,AU,-25.2406,130.9889,
271,koumac,NC,-20.5667,164.2833,
279,santa lucia,ES,27.9117,-15.5407,


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
coordinates = hotel_df[["Latitude", "Longitude"]]

In [26]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(coordinates, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))